In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tf_utils as utils

import tf_cnn as cnn
import cnn_utils
import tensorflow as tf

In [ ]:
generateResizedImages = False
reshuffleImages = False

if generateResizedImages:
    imresize.generate_resized_images(256)

if reshuffleImages:
    sourceFolders = ['E:\\Projects\\Hach2019\\Data\\ImageDataSetS256\\Plate',
        'E:\\Projects\\Hach2019\\Data\\ImageDataSetS256\\NonePlate']
    h5Target = 'E:\\Projects\\Hach2019\\Data\\ImageDataSetS256\\plate256.h5'
    dataset = utils.partition_dataset(sourceFolders, 0.8, 0.0, 0.2)
    utils.save_dataset_h5(target=h5Target, dataset=dataset)

    print("dataset shapes:")
    print("train_set_x: " + str(dataset["train_set_x"].shape))
    print("train_set_y: " + str(dataset["train_set_y"].shape))
    print("cv_set_x: " + str(dataset["cv_set_x"].shape))
    print("cv_set_y: " + str(dataset["cv_set_y"].shape))
    print("test_set_x: " + str(dataset["test_set_x"].shape))
    print("test_set_y: " + str(dataset["test_set_y"].shape))    
    print("classes: " + str(dataset["classes"].shape))

h5Source = 'E:\\Projects\\Hach2019\\Data\\ImageDataSetS256\\plate256.h5'
tr_set_x, tr_set_y, cv_set_x, cv_set_y, ts_set_x, ts_set_y, classes = utils.load_dataset_h5(source=h5Source)

tr_set_x_org = tr_set_x.copy()
tr_set_y_org = tr_set_y.copy()

tr_set_x = tr_set_x[:]
tr_set_y = tr_set_y[:, :]

print("loaded data shapes:")
print("tr_set_x: " + str(tr_set_x.shape))
print("tr_set_y: " + str(tr_set_y.shape))
print("ts_set_x: " + str(ts_set_x.shape))
print("ts_set_y: " + str(ts_set_y.shape))
print("cv_set_x: " + str(cv_set_x.shape))
print("cv_set_y: " + str(cv_set_y.shape))
print("classes: " + str(classes.shape))

In [ ]:
# show one of the images of class 0
index = 6
_, ax = plt.subplots()
ax.imshow(np.uint8(tr_set_x[index]))
print ("y = " + str(np.squeeze(tr_set_y[:, index])))
plt.title("Class =" + str(np.squeeze(tr_set_y[:, index])))
plt.show()

In [ ]:
import random
from PIL import Image, ImageDraw

def generate_random_image(width=256, height=256, scale=16):
    w = int(width/scale)
    h = int(height/scale)
    rand_pixels = [random.randint(0, 255) for _ in range(w * h * 3)]
    rand_pixels_as_bytes = bytes(rand_pixels)
    random_image = Image.frombytes('RGB', (w, h), rand_pixels_as_bytes)
    random_image = random_image.resize((width, height))
    return random_image

rand_img = generate_random_image()
_, ax = plt.subplots()
ax.imshow(np.uint8(rand_img))
plt.title("Class = 1")
plt.show()

In [ ]:
Modify = False
if Modify:
    start_idx = 108
    end_idx = tr_set_x.shape[0]
    width = tr_set_x.shape[1]
    height = tr_set_x.shape[2]
    print(start_idx, end_idx, width, height)
    for i in range(start_idx, end_idx):
        print('>', end="")
        tr_set_x[i] = generate_random_image(width=width, height=height, scale=32)

Good param config:
Cut-out: -80,
Scale: 32,
epochs: 30

In [ ]:
# show one of the images of class 1
index = 108
_, ax = plt.subplots()
ax.imshow(np.uint8(tr_set_x[index]))
plt.title("Class =" + str(np.squeeze(tr_set_y[:, index])))
plt.show()

In [ ]:
X_train = tr_set_x/255.
Y_train = utils.convert_to_one_hot(np.uint(tr_set_y), 2).T
X_test = ts_set_x/255.
Y_test = utils.convert_to_one_hot(np.uint(ts_set_y), 2).T 
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
print(tf.__version__)

tf.compat.v1.disable_eager_execution()

parameters, save_path = cnn.run_model(
    X_train, Y_train, X_test, Y_test,
    learning_rate=0.01, num_epochs=30, minibatch_size=64,
    print_cost=True)

In [ ]:
prediction, z3 = cnn.predict(X_test, parameters, save_path)
print("Z3:", z3)
print("Predictions:", prediction)
correct = np.sum(prediction == Y_test.T[1])
print("Accuracy:", correct/len(prediction))

for i in range(len(prediction)):
    title = "Class = " + str(np.squeeze(ts_set_y[:, i])) + "  Prediction = " + str(prediction[i])
    #_, ax = plt.subplots()
    #ax.imshow(np.uint8(ts_set_x[i]))
    #plt.title(title)
    #plt.show()

prediction, _ = cnn.predict(tr_set_x_org/255., parameters, save_path)
print("Predictions:", prediction)
correct = np.sum(prediction == tr_set_y_org)
print("Accuracy:", correct/len(prediction))

_, ax = plt.subplots()
ax.imshow(np.uint8(tr_set_x_org[128]))
plt.title(title)
plt.show()